In [173]:
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import xlrd
from openpyxl import load_workbook
import os
from datetime import datetime

In [174]:
characteristics_PR = ['rfid', 'subject', 'room', 'cohort', 'trial_id', 'drug', 'box','start_time', 'end_time', 
 'start_date', 'end_date', 'last_ratio', 'breakpoint', 'active_lever_presses', 'inactive_lever_presses',
 'reward_presses', 'reward_points']

In [175]:
coc_rewards = np.arange(19)
coc_lr = [0,1,2,4,6,9,12,15,20,25,32,40,50,62,77,95,118,145,178]
coc_lr_dict = dict(zip(coc_rewards,coc_lr))
coc_lr_dict

{0: 0,
 1: 1,
 2: 2,
 3: 4,
 4: 6,
 5: 9,
 6: 12,
 7: 15,
 8: 20,
 9: 25,
 10: 32,
 11: 40,
 12: 50,
 13: 62,
 14: 77,
 15: 95,
 16: 118,
 17: 145,
 18: 178}

In [176]:
oxy_rewards = np.arange(69)
oxy_lr = [0,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10] + np.arange(10,49).tolist() + [50,60,70,80,90,100,100,100,100,100]
oxy_lr_dict = dict(zip(oxy_rewards,oxy_lr))
oxy_lr_dict

{0: 0,
 1: 1,
 2: 1,
 3: 2,
 4: 2,
 5: 3,
 6: 3,
 7: 4,
 8: 4,
 9: 5,
 10: 5,
 11: 6,
 12: 6,
 13: 7,
 14: 7,
 15: 8,
 16: 8,
 17: 9,
 18: 9,
 19: 10,
 20: 10,
 21: 11,
 22: 12,
 23: 13,
 24: 14,
 25: 15,
 26: 16,
 27: 17,
 28: 18,
 29: 19,
 30: 20,
 31: 21,
 32: 22,
 33: 23,
 34: 24,
 35: 25,
 36: 26,
 37: 27,
 38: 28,
 39: 29,
 40: 30,
 41: 31,
 42: 32,
 43: 33,
 44: 34,
 45: 35,
 46: 36,
 47: 37,
 48: 38,
 49: 39,
 50: 40,
 51: 41,
 52: 42,
 53: 43,
 54: 44,
 55: 45,
 56: 46,
 57: 47,
 58: 48,
 59: 50,
 60: 60,
 61: 70,
 62: 80,
 63: 90,
 64: 100,
 65: 100,
 66: 100,
 67: 100,
 68: 100}

In [177]:
rfid_coc = pd.read_csv('/Users/yunyihuang/Desktop/gl_data/rfid_cocaine.csv', index_col=0)
rfid_oxy = pd.read_csv('/Users/yunyihuang/Desktop/gl_data/rfid_oxycodone.csv', index_col=0)

In [178]:
# get all the sheetnames in one excel workbook
def get_sheetnames_xlsx(file_name):
    wb = load_workbook(file_name, read_only=True, keep_links=False)
    return wb.sheetnames

# clean subject id
def clean_subject_id(sid):
    sid = sid.upper()
    if 'F' in sid:
        char = 'F'
    if 'M' in sid:
        char = 'M'
        
    idx = sid.index(char)
    return sid[idx:].split('.')[0]

# convert column names into correct format
def clean_cols(s):
    if 'U' in s:
        return s.replace('U','Active ')
    elif 'V' in s:
        return s.replace('V','Inactive ')
    elif 'Y' in s:
        return s.replace('Y','Reward ')
    else:
        return s

### COCAINE PR

In [179]:
input_path = '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc'
workbooks = [os.path.join(input_path, i) for i in sorted(os.listdir(input_path)) if ('modified_final' in i)]
workbooks

['/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc/C01GWASCocaine_allnonMEDPCsessions_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc/C02GWASCocaine_allnonMEDPCsessions_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc/C03GWASCocaine_allnonMEDPCsessions_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc/C04GWASCocaine_allnonMEDPCsessions_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc/C05GWASCocaine_allnonMEDPCsessions_modified_final.xlsx']

In [180]:
for wb in workbooks:
    worksheets = sorted(get_sheetnames_xlsx(wb))
    worksheets = [ws for ws in worksheets if 'PR' in ws]
    print(wb, len(worksheets))
    for ws in worksheets:
        print(ws)

/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc/C01GWASCocaine_allnonMEDPCsessions_modified_final.xlsx 3
C01HSPR01-20170814.xlsx
C01HSPR02-20170905.xlsx
C01HSPR03-20170908.xlsx
/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc/C02GWASCocaine_allnonMEDPCsessions_modified_final.xlsx 3
C02HSPR01-20171031.xlsx
C02HSPR02-20171122.xlsx
C02HSPR03-20171130.xlsx
/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc/C03GWASCocaine_allnonMEDPCsessions_modified_final.xlsx 3
C03HSPR01-20180220.xlsx
C03HSPR02-20180314.xlsx
C03HSPR03-20180319.xlsx
/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc/C04GWASCocaine_allnonMEDPCsessions_modified_final.xlsx 3
C04HSPR01-20180430
C04HSPR02-20180522
C04HSPR03-20180525
/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/allnonmedpc/C05GWASCocaine_allnonMEDPCsessions_modified_final.xlsx 3
C05HSPR01-20

In [181]:
for wb in workbooks:
    worksheets = sorted(get_sheetnames_xlsx(wb))
    worksheets = [ws for ws in worksheets if 'PR' in ws]
    print(len(worksheets))
    for ws in worksheets:
        print(ws)
        transform_old_pr(wb, ws, input_path)

3
C01HSPR01-20170814.xlsx
C01HSPR02-20170905.xlsx
C01HSPR03-20170908.xlsx
3
C02HSPR01-20171031.xlsx
C02HSPR02-20171122.xlsx
C02HSPR03-20171130.xlsx
3
C03HSPR01-20180220.xlsx
C03HSPR02-20180314.xlsx
C03HSPR03-20180319.xlsx
3
C04HSPR01-20180430
C04HSPR02-20180522
C04HSPR03-20180525
3
C05HSPR01-20180813
C05HSPR02-20180905
C05HSPR03-20180910


### OXYCODONE PR

In [143]:
input_path = '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Oxycodone/allnonmedpc'
workbooks = [os.path.join(input_path, i) for i in sorted(os.listdir(input_path)) if ('modified' in i)]
workbooks

['/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Oxycodone/allnonmedpc/C01GWASoxy_allnonMEDPCsessions_modified.xlsx']

In [144]:
wb = workbooks[0]
worksheets = sorted(get_sheetnames_xlsx(wb))
worksheets = [ws for ws in worksheets if 'PR' in ws]
worksheets

['C01HSOXYPR01-20180813',
 'C01HSOXYPR02-20180905',
 'C01HSOXYPR03T1-20180907',
 'C01HSOXYPR04T2-20180911',
 'C01HSOXYPR05T3-20180913',
 'C01HSOXYPR06T4-20180917']

In [109]:
ws = worksheets[0]
ws

'C01HSOXYPR01-20180813'

In [145]:
for wb in workbooks:
    worksheets = sorted(get_sheetnames_xlsx(wb))
    worksheets = [ws for ws in worksheets if 'PR' in ws]
    print(len(worksheets))
    for ws in worksheets:
        print(ws)
        transform_old_pr(wb, ws, input_path)

6
C01HSOXYPR01-20180813
C01HSOXYPR02-20180905
C01HSOXYPR03T1-20180907
C01HSOXYPR04T2-20180911
C01HSOXYPR05T3-20180913
C01HSOXYPR06T4-20180917


In [172]:
def transform_old_pr(wb,ws,input_path):
    filepath = os.path.join(input_path, wb)
    df_raw = pd.read_excel(filepath, sheet_name = ws).T.reset_index()

    # modify the header
    new_header = df_raw.iloc[0]     #grab the first row for the header
    df = df_raw[1:]                 #take the data except the header row
    df.columns = new_header 

    # get rid of 0s
    df.replace(0, np.nan, inplace=True)
    df.dropna(how='all', axis=1, inplace=True)
    df.fillna(0,inplace=True)
    
    # transform columns names
    ID_col = df.columns.tolist()[0]
    filtered_cols = [i for i in df.columns if i[0] in ['Y'] or 
                     i in [ID_col, 'Active Lever Presses', 'Inactive Lever Presses', 'Reward']]
    dff = df[filtered_cols]
    new_cols = [clean_cols(i) for i in dff.columns]
    dff.columns = new_cols
    
    # add extra info
    if 'OXY' in ws:
        drug = 'Oxycodone'
        parser = r'(\AC[0-9]{2})HS[OXY]*((?:PR)[0-9]+)'
        rfid_to_merge = rfid_oxy
        output_path = '/Users/yunyihuang/Desktop/gl_data/GEORGE/OXY_PR'
        lr_dict = oxy_lr_dict
    else:
        drug = 'Cocaine'
        parser = r'(C[0-9]{2})HS((?:PR)[0-9]{2})'
        rfid_to_merge = rfid_coc
        output_path = '/Users/yunyihuang/Desktop/gl_data/GEORGE/COC_PR'
        lr_dict = coc_lr_dict

    if '-' in ws:
            to_split = '-'
    if '_' in ws:
        to_split = '_'
        
    info, date = ws.split('.')[0].split(to_split)
    cohort, trial_id = re.findall(parser, ws)[0]
    dt = pd.to_datetime(date, format='%Y%m%d', errors='ignore')
    
    dff['room'] = ['N/A'] * len(dff)
    dff['cohort'] = [cohort[1:]] * len(dff)
    dff['trial_id'] = [trial_id] * len(dff)
    dff['drug'] = [drug] * len(dff)
    dff['box'] = [-999] * len(dff)
    dff['start_time'] = [datetime.min.time()] * len(dff)
    dff['end_time'] = [datetime.min.time()] * len(dff)
    dff['start_date'] = [dt] * len(dff)
    dff['end_date'] = [datetime.min.date()] * len(dff)
    
    # reorganize the columns
    colnames = dff.columns.tolist()
    reward_col_begin = colnames.index('Reward 0')
    reward_col_end = colnames.index('room')
    dff['Reward Points'] = dff.iloc[:, reward_col_begin:reward_col_end].values.tolist()
    dff.drop(dff.iloc[:, reward_col_begin:reward_col_end], inplace=True, axis=1)
    dff.rename(columns={"Reward": "Reward Presses", ID_col:"subject"}, inplace=True)
    
    dff['Reward Points'] = dff['Reward Points'].apply(lambda x: [i for i in x if i > 0])
    dff['Breakpoint'] = dff['Reward Points'].apply(lambda lst: lst[-1] if len(lst) > 0 else 0)
    dff['last_ratio'] = dff['Reward Presses'].apply(lambda x: lr_dict[x] if x in lr_dict else -999)
    
    dff = pd.merge(dff, rfid_to_merge,  how='left', on = ['subject'])
    dff.rename(columns=str.lower,inplace=True)
    dff.columns = dff.columns.str.replace(' ','_')
    dff.fillna({'rfid':-999}, inplace=True)
    dff = dff[characteristics_PR]
    
    filename = ws.split('.')[0]+'_transformed.csv'
    dff.to_csv(os.path.join(output_path, filename))

In [123]:
filename = ws.split('.')[0]+'_transformed.csv'
filename

'C01HSOXYPR01-20180813_transformed.csv'

In [124]:
dff.to_csv(os.path.join(output_path, filename))